<a href="https://colab.research.google.com/github/fininsight/nlp-deeplearning-tutorial/blob/master/2%EC%9E%A5_%EB%8B%A8%EC%96%B4%EC%9D%98_%ED%91%9C%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

원핫인코딩

sklearn에서 원핫인코딩의 입력값으로 숫자형이 들어가야 하기 때문에 

LabelEncoder를 사용하여 문자형 변수를 숫자형으로 변경해 준 후 원핫인코딩을 해준다. 

In [0]:
word_ls = ['원숭이','바나나','사과','코끼리']

from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()      
onehot_encoder = preprocessing.OneHotEncoder()    # 원핫인코딩

train_y = label_encoder.fit_transform(word_ls)    # 문자형 변수를 숫자형으로 변경
print(train_y)
# 원숭이 : 2 , 바나나 : 0 , 사과 : 1 , 코끼리 : 3 

train_y = train_y.reshape(len(train_y),1)
#print(train_y)

train_y = onehot_encoder.fit_transform(train_y)
train_y.toarray()



[2 0 1 3]


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([[0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.]])

Word2Vec

it_df : IT 뉴스 

culture_df : 문화 뉴스

전처리

In [0]:
# Jupyter Notebook 코드 
# Google drive에서 파일 불러오는것...ㅎㅎㅎ
# 데이터 불러오기
import pandas as pd
it_df = pd.read_csv('2_word_embedding_it.csv').dropna() # 디폴트로 NaN이 1개라도 포함되면 해당 행이 삭제
culture_df = pd.read_csv('2_word_embedding_culture.csv').dropna()

# word2vec을 하기 위한 전처리
# it_df와 culture_df의 컬럼 '1'의 토큰을 리스트 형태로 변경
it_token_ls = list(it_df.loc[:,'1']) 
culture_token_ls = list(culture_df.loc[:,'1'])

total_token_ls = it_token_ls + culture_token_ls  # it_df의 리스트와 culture_df의 리스트를 한개로 합침

# 리스트의 값을 ','로 분리
total_token_ls = [tokens.split(',') for tokens in total_token_ls]
print(total_token_ls[0][:10])
['배', '정회', '신임', '과학기술', '일자리', '진흥', '원장', '아시아', '경제', '이민']


gensim Word2Vec

In [0]:
# gensim이라는 패키지에 word2Vec 클래스 사용
from gensim.models import Word2Vec
word2vec = Word2Vec(
    sentences = total_token_ls,  # 학습시킬 문장
    size = 10,                  # 임베딩된 단어 벡터의 차원 크기
    alpha = 0.025,               # 학습률(Learning rate)
    min_count=2,                 # 2번 미만 등장한 단어는 제외
    window = 8,                  # 문맥의 크기 (window_size)
    sample = 0.001,              # sub-sampling
    sg = 1,                      # 0: CBOW, 1: Skip-gram
    iter = 10                    # 전체 문장 반복학습 횟수(epoch)
    )

# '인공'의 단어 임베딩 
word_embedding = word2vec.wv.__getitem__('인공')
print(word_embedding)


# '인공'과 유사도가 높은 단어
word_similar_1 = word2vec.wv.most_similar('인공')
print(word_similar_1)

#'데이터'와 유사도가 높은 단어
word_similar_2 = word2vec.wv.most_similar('데이터')
print(word_similar_2)


FastText

In [0]:
from gensim.models.word2vec import Text8Corpus
from gensim.models import FastText

ft_model = FastText(Text8Corpus('text.txt'),  
                                  size=10,                 # 임베딩된 단어 벡터의 차원 크기
                                  window=8,                 # 문맥의 크기 (window_size)
                                  min_count=2,            # 2번 미만 등장한 단어는 제외
                                  alpha = 0.025,                    # 학습률(Learning rate)
                                  sg = 1,                             # 0: CBOW, 1: Skip-gram
                                  iter = 10,              # 전체 문장 반복학습 횟수(epoch)
                                  min_n=3, max_n=6)            # 최소, 최대 Ngram 수   

# Getting most similar vectors
print(ft_model.wv.most_similar('인공'))


GloVe

In [0]:
!pip install glove_python

     |████████████████████████████████| 266kB 10.2MB/s 
  Created wheel for glove-python: filename=glove_python-0.1.0-cp36-cp36m-linux_x86_64.whl size=702674 sha256=c9477356dad73eab4bfe9a737b913b2916f1d1cd1a10052c2379e27f307d1793
  Stored in directory: /root/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python


In [0]:
from glove import Glove
from glove import Corpus

SyntaxError: ignored

In [0]:
corpus = Corpus() 
corpus.fit(result, window=5)
# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성

glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
# 학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 20.

NameError: ignored